In [ ]:
!pip install transformers datasets

In [ ]:
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install openpyxl
import pandas as pd

df = pd.read_excel('/content/drive/MyDrive/train.xlsx') # Use read_excel to read .xlsx files

In [ ]:
X = df['comment_text'].tolist()  # Convert to list for processing
y = df[df.columns[2:]].values

In [ ]:
X = df['comment_text'].tolist()  # Convert to list for processing
y = df[df.columns[2:]].values

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistil

In [ ]:
def preprocess_function(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='tf')

In [ ]:
def preprocess_function(texts):
    # Check if each element in 'texts' is a string
    valid_texts = [text for text in texts if isinstance(text, str)]
    return tokenizer(valid_texts, padding=True, truncation=True, return_tensors='tf')

In [ ]:
!pip install transformers
import tensorflow as tf
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification

# Assuming X and y are your original data
# and preprocess_function is defined as in your provided code

# Apply preprocessing, but keep track of the indices of valid texts
valid_indices = []
def preprocess_function(texts):
    valid_texts = [text for text in texts if isinstance(text, str)]
    valid_indices.extend([i for i, text in enumerate(texts) if isinstance(text, str)])  # Store indices of valid texts
    return tokenizer(valid_texts, padding=True, truncation=True, return_tensors='tf')

encoded_X = preprocess_function(X)

# Filter y to match the valid texts
y_filtered = y[valid_indices]

# Now create the tf.data.Dataset
tf_dataset = tf.data.Dataset.from_tensor_slices((dict(encoded_X), y_filtered))

In [ ]:
train_size = 0.7
val_size = 0.2
test_size = 0.1

train_dataset = tf_dataset.take(int(len(X) * train_size))
val_dataset = tf_dataset.skip(int(len(X) * train_size)).take(int(len(X) * val_size))
test_dataset = tf_dataset.skip(int(len(X) * (train_size + val_size)))

In [ ]:
train_dataset = train_dataset.batch(16).shuffle(1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(16).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(16).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),  # since logits are used
    metrics=['accuracy']
)

In [ ]:
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)


Epoch 1/10
447/447 [==============================] - 469s 955ms/step - loss: 0.1019 - accuracy: 0.9094 - val_loss: 0.0516 - val_accuracy: 0.9956
Epoch 2/10
447/447 [==============================] - 433s 968ms/step - loss: 0.0470 - accuracy: 0.9889 - val_loss: 0.0487 - val_accuracy: 0.9956
Epoch 3/10
447/447 [==============================] - 433s 968ms/step - loss: 0.0372 - accuracy: 0.9752 - val_loss: 0.0512 - val_accuracy: 0.9887
Epoch 4/10
447/447 [==============================] - 432s 966ms/step - loss: 0.0302 - accuracy: 0.9178 - val_loss: 0.0544 - val_accuracy: 0.9956
Epoch 5/10
447/447 [==============================] - 431s 965ms/step - loss: 0.0251 - accuracy: 0.8409 - val_loss: 0.0577 - val_accuracy: 0.9877
Epoch 6/10
447/447 [==============================] - 431s 965ms/step - loss: 0.0208 - accuracy: 0.9293 - val_loss: 0.0616 - val_accuracy: 0.7475
Epoch 7/10
447/447 [==============================] - 431s 965ms/step - loss: 0.0181 - accuracy: 0.5857 - val_loss: 0.0671 -

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

# Evaluate on the test set
for batch in test_dataset:
    X_true, y_true = batch
    yhat = model.predict(X_true)

    # Apply sigmoid and round predictions
    yhat = tf.nn.sigmoid(yhat.logits)
    yhat = tf.round(yhat)

    # Update metrics
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

# Print results
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}')

1/1 [==============================] - 0s 57ms/step
Precision: 0.7124183177947998, Recall: 0.26913580298423767, Accuracy: 0.9164990186691284
